## Inference
This notebook generates the labels for the unlabelled "test_dataset" using the pretrained model that was trained in "training.ipynb"

### Initialisation
Importing libraries and defining the device

In [ ]:
import torch
from torch import nn
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets
from PIL import Image
import pandas as pd
import torchvision.transforms.functional as IFU
import os

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

### The Model
Defining the model architecture and loading the pretrained weights

In [ ]:
# # Creating a CNN-based image classifier.
class ImageClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_layer_1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32)
        )
        self.conv_layer_2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64)
        )
        self.conv_layer_3 = nn.Sequential(
            nn.Conv2d(64, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(512)
        )

        self.conv_layer_4 = nn.Sequential(
          nn.Conv2d(512, 512, kernel_size=3, padding=1),
          nn.ReLU(),
          nn.BatchNorm2d(512))
        
        self.conv_layer_5 = nn.Sequential(
          nn.Conv2d(512, 256, kernel_size=3, padding=1),
          nn.ReLU(),
          nn.BatchNorm2d(256))

        self.max_pool = nn.MaxPool2d(2)
        
        self.avg_pool = nn.AvgPool2d(2)

        self.classifier = nn.Sequential(
          nn.Flatten(),
          nn.Linear(in_features=256*3*3, out_features=128),
          nn.ReLU(),
          nn.Dropout(0.1),
          nn.Linear(in_features=128, out_features=8),)
    def forward(self, x: torch.Tensor):
        x = self.conv_layer_1(x)
        x = self.max_pool(self.conv_layer_2(x))
        x = self.avg_pool(self.conv_layer_3(x))
        x = self.avg_pool(self.conv_layer_4(x))
        x = self.max_pool(self.conv_layer_4(x))
        x = self.max_pool(self.conv_layer_4(x))
        x = self.max_pool(self.conv_layer_4(x))
        x = self.conv_layer_5(x)
        x = self.classifier(x)
        return x
# Instantiate an object.
model = ImageClassifier().to(device)


In [ ]:
load = True #### SET TO TRUE IF YOU HAVE A PRESAVED CHECKPOINT
if load:
    checkpoint = torch.load("final_model_data.pt")
    model.load_state_dict(checkpoint['model_state_dict'])
    results = checkpoint['results']
    done_epochs = checkpoint['epoch']
else:
    results = None
    done_epochs = 0

done_epochs

### Forward Pass for a Single Image
Demonstrating the forward pass and inference by the model for one single image

In [ ]:
## Printing Model Summary

IMAGE_WIDTH = 200
IMAGE_HEIGHT = 200
# Install torchinfo if it's not available, import it if it is
try:
    import torchinfo
except:
    !pip install torchinfo
    import torchinfo

from torchinfo import summary
# do a test pass through of an example input size
summary(model, input_size=[1, 1, IMAGE_WIDTH ,IMAGE_HEIGHT])

In [ ]:
# Define transformation for images
IMAGE_SIZE = (200, 200)
data_transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor(),
])

In [ ]:
## Function to get image from filename
def get_image(filename, transform = None):
    image = Image.open(filename)
    if transform:
        image = transform(image)

    image = IFU.adjust_contrast(image, 1.8)
    return image

img_list = [] # List of paths to images
for i in range(2000):
    img_list.append(os.path.join("test_dataset", str(i+1)+".jpg"))

img = get_image(img_list[1491], data_transform) ## Selecting one random image

In [ ]:
plt.imshow(img.permute(1,2,0), cmap="Greys_r") ## Displaying the image
img.shape

In [ ]:
## Model prediction for the single image

model.eval()
with torch.inference_mode():
    pred = model(img.unsqueeze(1).to(device))

print(f"Output logits:\n{pred}\n")
print(f"Output prediction probabilities:\n{torch.softmax(pred, dim=1)}\n")
print(f"Output prediction label:\n{torch.argmax(torch.softmax(pred, dim=1), dim=1).item()}\n")

### Inference for all images

In [ ]:
## Generating prediction for all images in the test_dataset

predictions = []
for i in range(2000):
    image_path = img_list[i]
    img = get_image(image_path, data_transform)
    model.eval()
    with torch.inference_mode():
        pred = model(img.unsqueeze(1).to(device))

    label = torch.argmax(torch.softmax(pred, dim=1), dim=1)
    predictions.append(label.item())

In [ ]:
## Mapping prediction labels with their classes

label_to_int_map = {
    'bright dune': 0,
    'dark dune': 1,
    'spider': 2,
    'impact ejecta': 3,
    'slope streak': 4,
    'swiss cheese': 5,
    'crater': 6,
    'other': 7
}

int_to_label_map = {v:k for k, v in label_to_int_map.items()}

In [ ]:
## Building the final dataframe from the predictions and exporting it as a CSV file

img_names_series = pd.Series([str(i) + ".jpg" for i in range(1, 2001)])
pred_labels_series = pd.Series(predictions)
pred_label_names = pred_labels_series.replace(int_to_label_map)

out_df = pd.DataFrame({
    "File Name":img_names_series,
    "Class":pred_label_names
})

out_df.to_csv("the_crue_submission.csv", index=False)